In [3]:
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import os
from tqdm import tqdm # for loading progress

import seaborn as sns #  for plotting

from sklearn.svm import SVC,LinearSVC, LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import sys
sys.path.append('../elpv-dataset/utils')
from elpv_reader import load_dataset

ModuleNotFoundError: No module named 'elpv_reader'

In [190]:
from tqdm import tqdm

In [2]:
images, probas, types = load_dataset('./KOC/koc/data/labels.csv')

In [96]:
kp = sift.detect(images[0], None)

In [147]:
descriptors_list

[array([[ 30., 119.,  47., ...,   0.,   0.,   0.],
        [  8.,   7.,   0., ...,   3.,   0.,   0.],
        [  3.,   0.,   0., ...,  21., 137.,  36.],
        ...,
        [  0.,   0.,   0., ...,  22.,   2.,   0.],
        [  0.,   0.,   0., ...,   0.,   1.,  16.],
        [ 13.,   5.,   0., ...,   1.,   5.,   9.]], dtype=float32),
 array([[  9.,   8.,   0., ...,   3.,  10.,  21.],
        [  2.,   0.,   0., ...,  18., 137.,   0.],
        [  8.,   0.,   0., ...,   9., 140.,   1.],
        ...,
        [ 13.,   9.,   1., ...,   1.,   5.,  12.],
        [131.,  56.,  20., ...,   0.,   0.,   1.],
        [  0.,   0.,   0., ..., 128.,   2.,   0.]], dtype=float32),
 array([[  2.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,  79.,   2.,   0.,
           0.,   0.,   0.,   0.,  83.,  51.,   0.,   0.,  27.,   7.,   1.,
           1., 142.,   5.,   0.,   0., 123.,  36.,   1.,   1.,  43.,  58.,
           6.,   0.,   0.,   0.,   0.,   0.,   3., 142.,  34.,   6.,   4.,
           0.,   0.,   0.,  

In [164]:
from keras.applications.vgg19 import VGG19, preprocess_input

In [ ]:
# Get the KAZE keypoints

from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model

# detect keypoints with kaze
kaze = cv.KAZE_create()
#kp = kaze.detect(images[5])

image_keypoints = [kaze.detect(i) for i in tqdm(images, desc='Extracting keypoints using KAZE.')]

invalid_indices = [idx for idx, i in enumerate(image_keypoints) if i == ()]
new_y = np.delete(probas[:n_samples], invalid_indices)
filtered_images = np.delete(images, invalid_indices, axis=0)

vgg19 = VGG19(weights='imagenet')
model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('fc1').output)

In [185]:
image_keypoints = [i for i in image_keypoints if i != ()]

In [188]:
len(filtered_images)

2511

In [187]:
len(image_keypoints)

2511

In [191]:


# get the patch from the image using the keypoint
descriptors_list = []

for img, keypoints in tqdm(zip(filtered_images, image_keypoints), desc="Extracting descriptors from KAZE keypoints"):

    descriptors = np.array([])
    
    # Rotations of images aren't necessary since convolutions in VGG take care of orientation etc.
    for kp in keypoints:
        
        x, y = int(kp.pt[0]), int(kp.pt[1])
        size = int(kp.size)
        # Ensure coordinates and patch size are within the image boundaries
        x, y, size = max(x, 0), max(y, 0), min(size, min(img.shape[:2]))
        # Extract the patch around the keypoint
        #print(img.shape)
        #print('y:y+size, x:x+size', y,y+size, x,x+size)
        patch = img[y:y+size, x:x+size]

        # convert to 3 channel
        patch = cv.cvtColor(patch, cv.COLOR_GRAY2RGB)
        
        # Resize to 224x224 for VGG19 input
        patch = cv.resize(patch, (224, 224))
        # Convert patch to float and preprocess for VGG19
        patch = preprocess_input(image.img_to_array(patch).astype(np.float32))
        patch = np.expand_dims(patch, axis=0)
    
        descriptor = model.predict(patch, verbose=0)

        np.append(descriptors, descriptor)
    descriptors_list.append(descriptors)


all_descriptors = np.vstack(descriptors_list)

# upscale patch to 224x224 and rotate?

# feed each patch from a single image into VGG19 get the feature descriptor output

# use vlad to aggregate each descriptor for an image into a single global descriptor

# run the vlad vector through svm








Extracting descriptors from KAZE keypoints: 0it [00:00, ?it/s]

Extracting descriptors from KAZE keypoints: 1it [00:00,  5.64it/s]

Extracting descriptors from KAZE keypoints: 2it [00:00,  5.88it/s]

Extracting descriptors from KAZE keypoints: 3it [00:01,  1.32it/s]

Extracting descriptors from KAZE keypoints: 4it [00:02,  1.55it/s]

Extracting descriptors from KAZE keypoints: 5it [00:02,  1.90it/s]

Extracting descriptors from KAZE keypoints: 6it [00:06,  1.57s/it]

Extracting descriptors from KAZE keypoints: 7it [00:06,  1.16s/it]

Extracting descriptors from KAZE keypoints: 8it [00:06,  1.12it/s]

Extracting descriptors from KAZE keypoints: 9it [00:06,  1.51it/s]

Extracting descriptors from KAZE keypoints: 10it [00:08,  1.05s/it]

Extracting descriptors from KAZE keypoints: 11it [00:09,  1.21it/s]

Extracting descriptors from KAZE keypoints: 12it [00:09,  1.38it/s]

Extracting descriptors from KAZE keypoints: 13it [00:10,  1.13it/s]

Extracting descriptors from KAZE keypoints: 14

KeyboardInterrupt: 

In [152]:
a = np.array([])

In [154]:
all_descriptors.sha

array([[ 30., 119.,  47., ...,   0.,   0.,   0.],
       [  8.,   7.,   0., ...,   3.,   0.,   0.],
       [  3.,   0.,   0., ...,  21., 137.,  36.],
       ...,
       [  0.,   0.,   0., ...,  68.,  18.,   0.],
       [ 22.,   0.,   0., ...,   5.,  24.,  31.],
       [ 40.,   0.,   0., ...,   0.,   2.,  24.]], dtype=float32)

In [146]:
descriptors_list[0].shape

(10, 128)

In [123]:
kp[2].size

15.289507865905762

In [4]:
# Getting the descriptors with SIFT keypoints

n_samples = images.shape[0] # all training samples

sift = cv.SIFT_create()

# (rows: n_samples, columns: (s_i, 128)) where s is the number of keypoints detected for an image i.
descriptors_list = [sift.detectAndCompute(i, None)[1] for i in tqdm(images[:n_samples], desc="Extracting SIFT descriptors")]

# Any descriptors found which are non need to be removed from the y labels
invalid_indices = [idx for idx, i in enumerate(descriptors_list) if i is None]
new_y = np.delete(probas[:n_samples], invalid_indices)

# Remove the None values from the X data and convert the (n_samples, (s,128)) to (n_samples*sum(s_i: i in [1, n_samples], 128)
all_descriptors = np.vstack([i for i in descriptors_list if i is not None])




Extracting SIFT descriptors: 100%|█████████████████████████████████████████████████| 2624/2624 [00:40<00:00, 65.54it/s]
C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=6, random_state=0)

In [11]:
# Cluster the desciptors
k = 5
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(all_descriptors)

C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5, random_state=0)

In [12]:
# Vlad with power norm, l2 norm, PCA whitening
#from sklearn.decomposition import PCA

vlad_vectors = []
pca = PCA(whiten=True)  # You may need to set the number of components if required

# Compute VLAD vectors
for descriptors in descriptors_list:
    if descriptors is None:
        continue
    cluster_assignments = kmeans.predict(descriptors)
    vlad_vector = np.zeros((k, descriptors.shape[1]))  # (k, 128) vector

    for idx, cluster_idx in enumerate(cluster_assignments):
        vlad_vector[cluster_idx] += (descriptors[idx] - kmeans.cluster_centers_[cluster_idx])

    # Power normalization
    p = 0.5
    vlad_vector = np.sign(vlad_vector) * np.abs(vlad_vector) ** p
    
    # l2 Normalization
    vlad_vector = normalize(vlad_vector, norm='l2', axis=1)  # Normalize each row independently

    vlad_vectors.append(vlad_vector.flatten())

# Concatenate all VLAD vectors from all images to fit the PCA
all_vlad_vectors = np.vstack(vlad_vectors)

# PCA Whitening
all_vlad_vectors = pca.fit_transform(all_vlad_vectors)

# l2 Normalization post PCA
all_vlad_vectors = normalize(all_vlad_vectors, norm='l2', axis=1)

In [80]:
np.vstack(all_vlad_vectors).shape

(2610, 640)

In [102]:
# RUN svm classifier
X = np.vstack(vlad_vectors) # The descriptors as VLAD vectors
y = new_y # The y with all samples without any detected keypoints/descriptors removed

y_binary = np.where(y > 0, 1, 0)

# weights are:
# 1.0         (class 0) functional
# 0.33,0.67,1 (class 1) defective
# This is a binary classification problem
sample_weights = np.where((y == 0) | (y == 1), 1, y) 

X_train, X_test, y_train, y_test, sw_train, sw_test = train_test_split(
    X, y_binary, 
    sample_weights,   # the certainty of the class label
    test_size=0.25,   # from the paper 75/25 split
    random_state=42, 
    stratify=y_binary # ensure balanced functional/defective samples
  )

# Inverse proportion heuristic derived from King and Zeng (2001) from the paper
class_weights = {
    0: y_train.size / 2*np.sum(y_train == 0),
    1: y_train.size / 2*np.sum(y_train == 1)
}

# Initialize and train the SVM classifier
clf = LinearSVC(class_weight='balanced')
clf.fit(X_train, y_train, sample_weight=sw_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred, sample_weight=sw_test))

C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.79      0.77      0.78     377.0
           1       0.62      0.65      0.63 215.6666666666667

    accuracy                           0.73 592.6666666666667
   macro avg       0.71      0.71      0.71 592.6666666666667
weighted avg       0.73      0.73      0.73 592.6666666666667



In [103]:
?LinearSVC

Init signature:
LinearSVC(
    penalty='l2',
    loss='squared_hinge',
    *,
    dual='warn',
    tol=0.0001,
    C=1.0,
    multi_class='ovr',
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    verbose=0,
    random_state=None,
    max_iter=1000,
)
Docstring:     
Linear Support Vector Classification.

Similar to SVC with parameter kernel='linear', but implemented in terms of
liblinear rather than libsvm, so it has more flexibility in the choice of
penalties and loss functions and should scale better to large numbers of
samples.

The main differences between :class:`~sklearn.svm.LinearSVC` and
:class:`~sklearn.svm.SVC` lie in the loss function used by default, and in
the handling of intercept regularization between those two implementations.

This class supports both dense and sparse input and the multiclass support
is handled according to a one-vs-the-rest scheme.

Read more in the :ref:`User Guide <svm_classification>`.

Parameters
----------
penalty : {'l1

In [106]:
1e2

100.0

In [107]:
# SVM with optim
def svm_best_params(clf, X_train, X_val, y_train, y_val, sw_train, sw_val):
    param_grid = {
        'C': [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]
     
    }

    grid_search = GridSearchCV(estimator=clf,
                               param_grid=param_grid,
                               cv=5, # 5 fold cross-validation
                               n_jobs=-1,
                               verbose=2,
                               scoring='f1'
                              )
    grid_search.fit(X_train, y_train, sample_weight=sw_train)
    print('Best params:', grid_search.best_params_)
    
    # Validate the best parameters on the validation set
    best_model = grid_search.best_estimator_
    y_val_pred = best_model.predict(X_val)
    print("\nValidation Set Performance:")
    print(classification_report(y_val, y_val_pred, sample_weight=sw_val))

    return grid_search.best_params_

sample_weights = np.where((y == 0) | (y == 1), 1, y) 

# Split your data into a training+validation set and a test set
X_train_val, X_test, y_train_val, y_test, sw_train_val, sw_test = train_test_split(
    X, y_binary, 
    sample_weights, 
    test_size=0.25, 
    random_state=42, 
    stratify=y_binary
)

# Further split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, sw_train, sw_val = train_test_split(
    X_train_val, 
    y_train_val, 
    sw_train_val, 
    test_size=0.25,  # size of the validation set
    random_state=42, 
    stratify=y_train_val
)

# Use randomized search - faster
best_params = svm_best_params(LinearSVC(random_state=42, class_weight='balanced'), 
                             X_train, X_val, y_train, y_val, sw_train, sw_val)

svm_clf = LinearSVC(**best_params).fit(X_train_val, y_train_val, sample_weight=sw_train_val)

#rf_clf = RandomForestClassifier({'bootstrap': True, 'max_depth': 34, 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 394})

#rf_clf.fit(X_train_val, y_train_val, sample_weight=sw_train_val)

y_pred_rf = svm_clf.predict(X_test)

print(classification_report(y_test, y_pred_rf, sample_weight=sw_test))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Best params: {'C': 1.0}

Validation Set Performance:
              precision    recall  f1-score   support

           0       0.78      0.80      0.79     283.0
           1       0.63      0.60      0.62 161.00000000000003

    accuracy                           0.73     444.0
   macro avg       0.70      0.70      0.70     444.0
weighted avg       0.72      0.73      0.72     444.0

              precision    recall  f1-score   support

           0       0.79      0.82      0.80     377.0
           1       0.66      0.61      0.64 215.6666666666667

    accuracy                           0.74 592.6666666666667
   macro avg       0.72      0.72      0.72 592.6666666666667
weighted avg       0.74      0.74      0.74 592.6666666666667



In [19]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

In [18]:
# RUN svm regressor
X = np.vstack(vlad_vectors) # The descriptors as VLAD vectors
y = new_y # The y with all samples without any detected keypoints/descriptors removed

#y_binary = np.where(y > 0, 1, 0)

# weights are:
# 1.0         (class 0) functional
# 0.33,0.67,1 (class 1) defective
# This is a binary classification problem

#sample_weights = np.where((y == 0) | (y == 1), 1, y) 

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,   # from the paper 75/25 split
    random_state=42, 
    stratify=y # ensure balanced functional/defective samples
  )

# Inverse proportion heuristic derived from King and Zeng (2001) from the paper
"""class_weights = {
    0: y_train.size / 2*np.sum(y_train == 0),
    1: y_train.size / 2*np.sum(y_train == 1)
}"""

# Initialize and train the SVM classifier
clf = LinearSVR()
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Print the classification report
#print(classification_report(y_test, y_pred))

C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ajval\Documents\COMP9517_Computer_Vision\labenv\venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [21]:
mean_squared_error(y_test, y_pred)

0.16446825591423328

In [24]:
categories = np.unique(probas)

In [31]:
np.abs(y_pred[:, None] - categories[None, :]).argmin(axis=1)

array([0, 2, 1, 2, 2, 0, 0, 1, 0, 0, 0, 0, 3, 1, 3, 1, 2, 0, 0, 2, 3, 3,
       3, 1, 1, 0, 0, 2, 0, 2, 0, 0, 0, 1, 2, 0, 2, 0, 1, 3, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 3, 0, 0, 1, 1, 3, 0, 1, 0, 1, 2,
       1, 0, 2, 2, 0, 1, 0, 2, 1, 2, 1, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 3, 0, 0, 1, 0, 2, 1, 2, 1, 1, 2, 1, 0, 1, 0, 0, 1, 1, 2,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 2, 1, 1, 0, 1, 0, 1, 1, 2, 0, 0, 2, 1,
       3, 2, 1, 1, 1, 3, 3, 2, 3, 1, 1, 1, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0,
       1, 0, 1, 2, 3, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2,
       1, 1, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 2, 0, 0, 1, 0, 1, 2, 2, 1, 1, 0, 1, 1, 1, 3, 2, 0, 0, 1, 0, 1,
       0, 1, 1, 3, 0, 0, 1, 3, 1, 0, 3, 1, 2, 1, 0, 2, 1, 1, 0, 2, 0, 2,
       0, 0, 1, 1, 1, 1, 3, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 2, 0,
       0, 0, 0, 3, 1, 3, 0, 1, 0, 3, 3, 2, 1, 0, 3, 1, 1, 1, 0, 0, 2, 0,
       0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 0, 1, 0, 0,

In [38]:
np.abs(y_test[:, None] - categories[None, :]).argmin(axis=1)

array([0, 0, 2, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 0, 3, 1, 0, 2, 0, 3, 1, 3,
       3, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 3, 3, 0, 3, 0, 0, 3, 0, 1, 2, 3,
       0, 3, 0, 0, 0, 0, 1, 0, 3, 3, 0, 3, 0, 0, 0, 3, 3, 2, 3, 0, 3, 3,
       1, 1, 3, 2, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0, 1, 1, 0,
       0, 0, 0, 3, 3, 0, 3, 0, 2, 3, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 3, 0, 3, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 3, 3,
       3, 0, 3, 0, 2, 0, 3, 3, 0, 2, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 2, 3, 3, 0, 0, 0, 0, 0, 1, 3, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 3, 0, 0, 0, 1, 3,
       0, 3, 0, 0, 2, 0, 1, 0, 3, 3, 0, 0, 0, 0, 3, 3, 1, 0, 0, 0, 0, 3,
       0, 0, 1, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 3, 0, 3, 3, 0, 1, 3, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 3, 3, 1, 1, 0, 3, 3, 3, 3, 0, 3, 0, 0, 1, 0, 0, 3, 0,
       0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 0, 1,

In [40]:
print(classification_report(np.abs(y_test[:, None] - categories[None, :]).argmin(axis=1), np.abs(y_pred[:, None] - categories[None, :]).argmin(axis=1)))

              precision    recall  f1-score   support

           0       0.79      0.57      0.66       377
           1       0.14      0.43      0.21        74
           2       0.05      0.19      0.08        27
           3       0.82      0.29      0.42       175

    accuracy                           0.46       653
   macro avg       0.45      0.37      0.34       653
weighted avg       0.69      0.46      0.52       653



In [42]:
np.bincount(y_train)

array([1131,  826], dtype=int64)

In [57]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint

def rf_rand_best_params(rf_clf, X_train, X_val, y_train, y_val, sw_train, sw_val):
    param_dist = {
        'n_estimators': randint(100, 500),  # Number of trees, sampled from a uniform distribution
        'max_depth': [None] + list(randint(10, 50).rvs(3)),  # Maximum depth of the tree
        'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
        'min_samples_leaf': randint(1, 10),   # Minimum number of samples required at each leaf node
        'bootstrap': [True, False]            # Method of selecting samples for training each tree
    }

    rand_search = RandomizedSearchCV(estimator=rf_clf,
                                     param_distributions=param_dist,
                                     n_iter=100,           # Number of parameter settings that are sampled
                                     cv=5,                 # Number of folds in cross-validation
                                     verbose=2,
                                     random_state=42,
                                     n_jobs=-1,            # Number of jobs to run in parallel
                                     scoring='accuracy')   # Scoring metric to optimize
    
    # Perform the random search on the training data
    rand_search.fit(X_train, y_train, sample_weight=sw_train)
    
    print("Best parameters found: ", rand_search.best_params_)

    return rand_search.best_params_
    

def rf_best_params(rf_clf, X_train, X_val, y_train, y_val, sw_train, sw_val):
    param_grid = {
       'n_estimators': [100, 200, 300],  # Number of trees in the forest
        'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
        'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
        #'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required at each leaf node
        #'bootstrap': [True, False]        # Method of selecting samples for training each tree  
    }

    grid_search = GridSearchCV(estimator=rf_clf,
                               param_grid=param_grid,
                               cv=3, # 5 fold cross-validation
                               n_jobs=-1,
                               verbose=2,
                               scoring='accuracy'
                              )
    grid_search.fit(X_train, y_train, sample_weight=sw_train)
    print('Best params:', grid_search.best_params_)
    
    # Validate the best parameters on the validation set
    best_model = grid_search.best_estimator_
    y_val_pred = best_model.predict(X_val)
    print("\nValidation Set Performance:")
    print(classification_report(y_val, y_val_pred, sample_weight=sw_val))

    return grid_search.best_params_
    

In [58]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

X = np.vstack(vlad_vectors) # The descriptors as VLAD vectors
y = new_y # The y with all samples without any detected keypoints/descriptors removed

y_binary = np.where(y > 0, 1, 0)

# weights are:
# 1.0         (class 0) functional
# 0.33,0.67,1 (class 1) defective
# This is a binary classification problem
sample_weights = np.where((y == 0) | (y == 1), 1, y) 

"""X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, 
    test_size=0.25,   # from the paper 75/25 split
    random_state=42, 
    stratify=y_binary # ensure balanced functional/defective samples
  )"""

# Split your data into a training+validation set and a test set
X_train_val, X_test, y_train_val, y_test, sw_train_val, sw_test = train_test_split(
    X, y_binary, 
    sample_weights, 
    test_size=0.25, 
    random_state=42, 
    stratify=y_binary
)

# Further split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, sw_train, sw_val = train_test_split(
    X_train_val, 
    y_train_val, 
    sw_train_val, 
    test_size=0.25,  # size of the validation set
    random_state=42, 
    stratify=y_train_val
)

# Initialize the Random Forest classifier
# You can specify 'balanced' or 'balanced_subsample' for the class_weight parameter
# if your classes are imbalanced
"""rf_clf = RandomForestClassifier(n_estimators=100,  # The number of trees in the forest
                                random_state=42,
                                class_weight='balanced')  # or class_weights if you calculated them manually"""

# Use Grid search - slower
"""best_params = rf_best_params(RandomForestClassifier(random_state=42, class_weight='balanced'), 
                             X_train, X_val, y_train, y_val, sw_train, sw_val)"""

# Use randomized search - faster
best_params = rf_rand_best_params(RandomForestClassifier(random_state=42, class_weight='balanced'), 
                             X_train, X_val, y_train, y_val, sw_train, sw_val)

#rf_clf = RandomForestClassifier(**best_params)
rf_clf = RandomForestClassifier({'bootstrap': True, 'max_depth': 34, 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 394})

rf_clf.fit(X_train_val, y_train_val, sample_weight=sw_train_val)

y_pred_rf = rf_clf.predict(X_test)

print(classification_report(y_test, y_pred_rf, sample_weight=sw_test))


Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [72]:
# Split your data into a training+validation set and a test set
X_train_val, X_test, y_train_val, y_test, sw_train_val, sw_test = train_test_split(
    X, y_binary, 
    sample_weights, 
    test_size=0.25, 
    random_state=42, 
    stratify=y_binary
)

# Further split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, sw_train, sw_val = train_test_split(
    X_train_val, 
    y_train_val, 
    sw_train_val, 
    test_size=0.25,  # size of the validation set
    random_state=42, 
    stratify=y_train_val
)

rf_clf = RandomForestClassifier(**{'bootstrap': True, 'max_depth': 34, 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 394}, random_state=10)

rf_clf.fit(X_train_val, y_train_val, sample_weight=sw_train_val)

y_pred_rf = rf_clf.predict(X_test)

print(classification_report(y_test, y_pred_rf, sample_weight=sw_test))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83     377.0
           1       0.73      0.60      0.66 215.6666666666667

    accuracy                           0.77 592.6666666666667
   macro avg       0.76      0.74      0.74 592.6666666666667
weighted avg       0.77      0.77      0.77 592.6666666666667



In [73]:
from sklearn.model_selection import KFold, StratifiedKFold

# Define the KFold cross validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_results = []

# Perform KFold cross-validation
for train_index, test_index in skf.split(X_train_val, y_train_val):
    # Split the data
    X_train_fold, X_test_fold = X_train_val[train_index], X_train_val[test_index]
    y_train_fold, y_test_fold = y_train_val[train_index], y_train_val[test_index]
    sw_train_fold, sw_test_fold = sw_train_val[train_index], sw_train_val[test_index]
    
    # Train the model
    #rf_clf = RandomForestClassifier()
    rf_clf = RandomForestClassifier(**{'bootstrap': True, 'max_depth': 34, 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 394}, random_state=10)
    
    rf_clf.fit(X_train_fold, y_train_fold, sample_weight=sw_train_fold)
    
    # Predict on the test fold
    y_pred_fold = rf_clf.predict(X_test_fold)
    
    # Evaluate the model
    fold_result = classification_report(y_test_fold, y_pred_fold, sample_weight=sw_test_fold, output_dict=True)
    fold_results.append(fold_result)
    
    # Optionally print the classification report for each fold
    print(f"Classification report for fold:\n{classification_report(y_test_fold, y_pred_fold, sample_weight=sw_test_fold)}")

# Calculate the average performance over all folds
# This is a simple way to do it by averaging the scores
# You might want to store individual scores and compute a weighted average if the folds are of different sizes
average_performance = {}
for key in fold_results[0].keys():
    average_performance[key] = sum(d[key] for d in fold_results) / len(fold_results)

# Print the average performance
print("Average performance across all folds:", average_performance)

Classification report for fold:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83     227.0
           1       0.74      0.60      0.66 134.6666666666666

    accuracy                           0.77 361.66666666666663
   macro avg       0.77      0.74      0.75 361.66666666666663
weighted avg       0.77      0.77      0.77 361.66666666666663

Classification report for fold:
              precision    recall  f1-score   support

           0       0.78      0.84      0.81     226.0
           1       0.66      0.58      0.62 126.99999999999996

    accuracy                           0.74 352.99999999999994
   macro avg       0.72      0.71      0.71 352.99999999999994
weighted avg       0.74      0.74      0.74 352.99999999999994

Classification report for fold:
              precision    recall  f1-score   support

           0       0.77      0.88      0.82     226.0
           1       0.72      0.55      0.63 130.99999999999994

    ac

TypeError: unsupported operand type(s) for +: 'int' and 'dict'

In [76]:
X_train_val.shape

(1957, 640)

In [75]:
rf_clf.feature_importances_.shape

(640,)

In [68]:
fold_results

[{'0': {'precision': 0.8310810810810807,
   'recall': 0.8686440677966102,
   'f1-score': 0.8494475138121544,
   'support': 236.0},
  '1': {'precision': 0.6990291262135924,
   'recall': 0.6334310850439886,
   'f1-score': 0.6646153846153848,
   'support': 113.66666666666663},
  'accuracy': 0.7921830314585318,
  'macro avg': {'precision': 0.7650551036473365,
   'recall': 0.7510375764202994,
   'f1-score': 0.7570314492137696,
   'support': 349.66666666666663},
  'weighted avg': {'precision': 0.7881547544749667,
   'recall': 0.792183031458532,
   'f1-score': 0.7893638569426612,
   'support': 349.66666666666663}},
 {'0': {'precision': 0.756722151088348,
   'recall': 0.8794642857142857,
   'f1-score': 0.8134893324156915,
   'support': 224.0},
  '1': {'precision': 0.7254237288135594,
   'recall': 0.52970297029703,
   'f1-score': 0.6123032904148786,
   'support': 134.66666666666663},
  'accuracy': 0.7481412639405204,
  'macro avg': {'precision': 0.7410729399509537,
   'recall': 0.70458362800565

In [77]:
X_train.shape

(1467, 640)

In [82]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier



X = np.vstack(vlad_vectors) # The descriptors as VLAD vectors
y = new_y # The y with all samples without any detected keypoints/descriptors removed

y_binary = np.where(y > 0, 1, 0)

# weights are:
# 1.0         (class 0) functional
# 0.33,0.67,1 (class 1) defective
# This is a binary classification problem
sample_weights = np.where((y == 0) | (y == 1), 1, y) 

X_train, X_test, y_train, y_test, sw_train, sw_test = train_test_split(
    X, y_binary,
    sample_weights,
    test_size=0.25,   # from the paper 75/25 split
    random_state=42, 
    stratify=y_binary # ensure balanced functional/defective samples
  )

gb_clf = GradientBoostingClassifier().fit(X_train, y_train, sample_weight=sw_train)
#gb_clf.score(X_test, y_test)

y_pred = gb_clf.predict(X_test)

print(classification_report(y_test, y_pred, sample_weight=sw_test))

[[320.          57.        ]
 [ 97.         118.66666667]]
